# Attaque temporelle sur OpenSSH

En 2016, une faille sur le logiciel OpenSSH, un serveur SSH très courant sur les environnements Unix, a été révélée publiquement et identifiée comme CVE-2016-6210. La faille en question est exploitable via une attaque temporelle sur la phase d'authentification et donne accès par ce canal auxiliaire à la liste des utilisateurs du serveur. Pour les besoins du TP, la faille simplifiée a été réimplémentée dans le module `ssh.py`, qui contient une fonction `authenticate` et qui s'utilise comme dans l'exemple ci-dessous :

In [ ]:
from ssh import authenticate

# Two servers are available: 'vulnerable' (which is vulnerable to CVE-2016-6210) and 'secured' (which has been patched).

# Try to connect and to authenticate on a vulnerable remote server with given credentials
print(authenticate(server='vulnerable', user='test', password='test'))

# Try to connect and to authenticate on a secured remote server with given credentials
print(authenticate(server='secured', user='test', password='test'))

### Question 1
Écrire un script qui donne le temps de réponse (en millisecondes) de la connexion SSH pour les identifiants suivants :
- `user`: "root", `password`: "wrong"
- `user`: "unknown_user", `password`: "wrong"

### Question 2
Écrire une fonction `chrono` qui, à partir d'un `user` et d'un `password` donné, mesure le temps de réponse de l'authentification sur `server` en millisecondes.

In [6]:
def chrono(server, user, password):
    pass

### Question 3
Mettre en évidence l'influence (ou non) de la taille du `password` sur les identifiants `root` et `unknown_user` pour les deux serveurs (vulnerable et sécurisé) :

### Question 4
Écrire une fonction `check_user` qui teste si un utilisateur `user` existe ou non sur le serveur vulnérable :

In [8]:
def check_user(user):
    pass

### Question 5
Tenter d'extaire une liste d'utilisateurs du serveur vulnérable à partir de la liste des utilisateurs répandus sur des serveurs Unix en pièce jointe `users.txt` :